## -------------------------------------------------------------------------
## YOUR SUBMISSION WILL NOT BE RECORDED UNLESS YOU COMPLETE YOUR DETAILS HERE
### *replace the text in the brackets below with your own details*

### NAME: [First name, Last name]
### EMAIL: [email address that is registered with Wheeler]

## -------------------------------------------------------------------------

## Simplyfied Replication of Storeygard (2016) 

In this assignment you have to generate the (simplyfied) final dataset used in **"Farther on down the road: transport costs, trade and urban growth"** (2016) Review of Economic Studies 83(3): 1263-1295. For this assignment, we are going to focus on **Tanzania** for the year **1992** and **2013**.

The final dataset (6 columns, 148 raws) should contained:
   - 74 Tanzanian city names (repeated twice for 1992 and 2013)
   - city populations (same value for 1992 and 2013)
   - a column for the year (1992 and 2013)
   - light values in both years (1992 and 2013)
   - oil prices in both years (1992 and 2013) 
   - distance to the primary city (same value for 1992 and 2013)

The objective is to use the geoprocessing tools and routines seen in class to solve the intermediate steps needed to assemble the final dataset. 

We have organized the assigment into 6 blocks:

1. Identify Tanzanian Cities
2. Compute total light emitted by each city in 1992 and 2013
3. Identify Tanzania's primary city
4. Create a routable road network
5. Find optimal paths between cities
6. Build the final dataset

Each section and subsection provide a detailed explanation of

- Inputs: shapefiles, raster
- Task objective and description
- Output

We are providing the packages we exepect you to use as well as the Coordinate Reference Systems.

Good luck!


In [ ]:
%pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import rasterio
from rasterstats import zonal_stats
import matplotlib.pyplot as plt
from shapely.ops import nearest_points
from shapely.geometry import LineString, Point
import networkx as nx

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
World=gpd.read_file('/content/drive/MyDrive/GIS_Research_2023/2/data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp')

In [ ]:
dir_path='/content/drive/MyDrive/GIS_Research_2023/Assignment'

In [ ]:
# Defining Coordinate Reference System to use throughout the whole notebook
wgs84_crs = CRS.from_string('EPSG:4326') # WGS 1984
aeaa_crs = CRS.from_string("esri:102022") # albers equal area africa crs
aedc_crs = CRS.from_string("esri:102023") # Africa equidistant conic

# 1. Identify Tanzanian cities

The unit of observations is the city. Later on, we would to like to calculate average luminosity in both 1992 and 2013 for each city. To do that we need to have a polygon associated with each city. We need to find a way to assign a city name to its corresponding polygon.

- We have provided point locations of cities around the world (`citypop_v4_latlons.csv`) and their maximal night lights extents (`city_extents.shp`)

- Write code to identify, in Tanzania, light extents with known cities and discard light extends that cannot be identified with a city




In [ ]:
#import necessary libraries 
import geopandas as gpd
import pandas as pd


In [ ]:
#Load city locations data
df=pd.read_csv(dir_path+'/data/citypop_v4_latlons.csv')

In [ ]:
#Load city extents shapefile
city_extents_gdf=gpd.read_file(city_extents.shp)

In [ ]:
#Filter extents to Tanzania only
tanzania_extents_gdf=city_extents_gdf[city_extents_gdf['COUNTRY']=='Tanzania']

In [ ]:
#Join city locations data with extents on city name
tanzania_extents_gdf=tanzania_extents_gdf.join(cities_df.set_index('City'),on='city_name')

In [ ]:
#Drop extents with no matching city
tanzania_cities_extents_gdf.dropna(subset=['City'],inplace=True)

In [ ]:
#Export filtered extents to new shapefile

In [ ]:
tanzania_extents_gdf.to_file('tanzania_city_extents.shp')

# 2. Compute total light emitted by each city in 1992 and 2013

Now that you manage to assign each city point to a polygon, your task is to calculate the average luminosity level for each polygon for the years 1992 and 2013.

- We have provided raster data of nighttime lights in Tanzania for 1992 and 2013 (`tza_lights_1992.tif` and `tza_lights_2013.tif`). 

- Combine them with the identified city extents above to compute total light emitted by each city in both years

- As a sanity check, make scatterplots of population against night lights (use log scale for better results). Hint: the two should be positevely correlated!

Please make scatterplots of population against night lights (in log scale) below. 

# 3. Identify Tanzania's primary city

To make things simple, we call the primary city the one with the largest population. 

- Find the primary city.

- Plot the primary city with one color
- Plot all cities except the primary with a different color

- Add the outline of Tanzania to the plot (this is available in the file `tza_poly/tza_polygon.shp` we have provided for you)

# 4. Create a routable road network

As part of the identification stratey in Storeygard (2016), we need to compute the distance between each city and the primary city following the road network. In this section we will prepare the network, in section 5 we will solve it with the Djkstra's algorithm for the least costly path. 

The network should have the following elements: i) city points (as nodes), ii) roads (as edges), iii) connectors from city to roads (as edges)

- Using the file `roads.shp` we have provided for you as well as the point locations of identified cities in Section 1, create a network dataset that you can use to find shortest paths

- While this is somewhat involved, you have seen this exact procedure in the lecture on Donaldson and Hornbeck's paper on the American railroad. The steps we show there are what is needed here: you just need to adapt the code to fit the inputs for this application!

- For the costs, you can assume the following parameters (these relative costs are made up):
    - `cost_per_km_paved` = 1
    - `cost_per_km_unpaved` = 4
    
- For the connector pieces connecting city points to the road network, you can assume that they are paved

## 4.1 Create Road connectors to the cities points

In this subsection, you will generate the connectors from the cities to the closest road segment

## 4.2 Deal with connectivity issues

As discussed in class, drawing the connectors is not sufficient to ensure full connectivity due to numerical precision. 

- Rescale the connectors by a small factor. Hint: There is one city connector that needs an extra large rescaling!


Remember that after you applied the rescaling you also need to snap the cities' location to the rescaled connectors. This is achieved by the function below

In [ ]:
def snap_points_to_modified_connectors(points_gdf, connectors_gdf):
    
    out = points_gdf.copy()
    
    for index, row in points_gdf.iterrows():
        
        tmp_gdf = connectors_gdf.copy()
        tmp_gdf['distance'] = tmp_gdf.distance(row['geometry'])
        closest_geom = list(tmp_gdf.sort_values('distance')['geometry'])[0]
        closest_pts = [Point(coords) for coords in closest_geom.coords]
        dists = [row['geometry'].distance(x) for x in closest_pts]
        minimum = min(dists)
        minimum_index = [i for i, j in enumerate(dists) if j == minimum][0]      
        out.loc[index, 'geometry'] = closest_pts[minimum_index]
        
    return out

Use the function above, to snap the identified city to the rescale connectors

## 4.3 Join roads and connectors

Now you can join the roads and the rescaled connectors to finalize the construction of your network edges

## 4.4 Examine connectivity

Now we can examine the connectivity of the network you created. 

Remember that to build the network you need to define the following function

In [ ]:
# taken from here:
# https://www.reddit.com/r/gis/comments/b1ui7h/geopandas_how_to_make_a_graph_out_of_a/

def gdf_to_nx(gdf_network):
    # generate graph from GeoDataFrame of LineStrings
    net = nx.Graph()
    net.graph['crs'] = gdf_network.crs
    fields = list(gdf_network.columns)

    for index, row in gdf_network.iterrows():
        first = row.geometry.coords[0]
        last = row.geometry.coords[-1]

        data = [row[f] for f in fields]
        attributes = dict(zip(fields, data))
        net.add_edge(first, last, **attributes)

    return net

Build the network!

Also generate a geodataframe with the number of connected component for each city. [again check carefully the network notebook for Donalson and Hornbeck's replication]

Now plot it! 

If you see an unconnected city (number of components=2), then go back to Section 4.2 and inspect the data better!

## 4.5 Add costs

Now you are ready to assign the costs to paved and unpaved roads.

# 5. Find optimal paths between cities

Now that your network is routable, fully connected, and costs are specified, you can solve it!

- Using the roads and cities network created in step 4, solve the network to find the shortest paths
- Once again, you should follow the code we have provided during the lecture on Donaldson and Hornbeck's paper on the American railroad

- Now iterate over the list of centroids to grab the elements from the dictionary and insert them into a list of lists into the right order. 
- Finally, convert the  list of lists into a `pandas` dataframe. 

Notice that for this application we would just need the distance of each city to the primary city!

- Create a dataframe having 2 columns: i) city names and ii) distance to primary city.

# 6. Build the final dataset

- The point of this data construction exercise is to build a dataset you can use to run Storeygard's regression.

- You should therefore build this dataset as a final exercise

- The dataset should have
    - city names
    - city populations
    - a column for the year
    - lights in both years
    - oil prices in both years (we have provided the oil price data in `Europe_Brent_Spot_Price_FOB.csv`)
    - distance to the primary city